In [ ]:
from neuro_evolution_controller import main
import matplotlib.pyplot as plt
import mujoco
import numpy as np
from mujoco import viewer


x, a, gene = None, None, None
if __name__ == "__main__":
    x, a, gene = main(time=30, population=10, generations=20)
    plt.figure(figsize=(10, 6))
    plt.plot(a, 'm-', label='Distance to Target')

    plt.ylim(bottom = 0)
    plt.legend()
    plt.grid(True)

In [ ]:
# import random
# if __name__ == "__main__":
#     runs = 2
#     generations = 30
#     all_best = []

#     for r in range(runs):
#         # (optioneel) reproduceerbaar maken
#         seed = 1234 + r
#         np.random.seed(seed)
#         random.seed(seed)

#         _, f_best = main(time=30, population=10, generations=generations)
#         all_best.append(np.asarray(f_best, dtype=float))

#     # Lengtes gelijk trekken (voor de zekerheid)
#     min_len = min(len(b) for b in all_best)
#     A = np.vstack([b[:min_len] for b in all_best])   # shape: (runs, min_len)

#     mean = A.mean(axis=0)
#     std  = A.std(axis=0, ddof=1)                     # sample std
#     gens = np.arange(min_len)                        # of np.arange(1, min_len+1)

#     plt.figure(figsize=(10, 6))
#     plt.plot(gens, mean, 'm-', label='Mean best per generation')
#     plt.fill_between(gens, mean - std, mean + std, alpha=0.2, label='±1 SD')
#     plt.xlabel('Generation')
#     plt.ylabel('Distance to goal (lower is better)')
#     plt.ylim(bottom=0)
#     plt.legend()
#     plt.grid(True)
#     plt.tight_layout()
#     plt.show()

In [ ]:
from pathlib import Path
from networkx import DiGraph
from networkx.readwrite import json_graph
import json

def load_graph_from_json(
    load_file: Path | str,
) -> DiGraph:
    with Path(load_file).open("r", encoding="utf-8") as f:
        data = json.load(f)

    graph = json_graph.node_link_graph(data, edges="edges", directed=True)
    return graph

In [ ]:
from ariel.body_phenotypes.robogen_lite.constructor import (
    construct_mjspec_from_graph,
)
from ariel.simulation.environments.simple_flat_world import SimpleFlatWorld
from neuro_evolution_body import DATA
from neuro_evolution_controller import nn_control

mujoco.set_mjcb_control(None) # DO NOT REMOVE
world = SimpleFlatWorld()
robot_graph = load_graph_from_json(DATA / "robot_graph.json")
gecko_core= construct_mjspec_from_graph(robot_graph)

world.spawn(gecko_core.spec, spawn_position=[0, 0, 0])
model = world.spec.compile()
data = mujoco.MjData(model) # type: ignore
geoms = world.spec.worldbody.find_all(mujoco.mjtObj.mjOBJ_GEOM)
to_track = [data.bind(geom) for geom in geoms if "core" in geom.name]

input_size  = len(data.qvel)  # e.g., 6 positions + 6 velocities
hidden_size = 16
output_size = model.nu
history = []

mujoco.set_mjcb_control(lambda m,d: nn_control(m, d, to_track, np.array(x[-1]), history, input_size, hidden_size, output_size))


viewer.launch(
    model=model,  # type: ignore
    data=data,
)